In [1]:
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm.auto import tqdm
from scipy.optimize import minimize
from scipy.interpolate import RegularGridInterpolator
import imageio.v3 as iio

In [2]:
SRC = '/Users/gimli/cvr/data/zavity/Cely_sken_3_110proc'

In [3]:
videos = [file for file in os.listdir(SRC) if file.endswith("MP4")]

In [4]:
videos

In [5]:
video_timing = []
for video in videos:
    # taken from palivo - list timing of the video
    _cmd = ["ffprobe",
        "-i", str(os.path.join(SRC, video)),
        "-show_frames",
        "-show_entries", "frame=pkt_dts_time",
        "-select_streams", "v:0",
        "-print_format", "flat"]
    video_info_file_path = os.path.join(SRC, video.replace(".MP4", "-video-info.txt"))
    if not os.path.isfile(video_info_file_path):    
        with open(video_info_file_path, 'w') as f:
            p1 = subprocess.call(_cmd, stdout=f)
    with open(video_info_file_path, "rt") as f:
        video_timing.append(f.readlines())

In [6]:
timings = []
for vt in video_timing:
    timing = []
    for frame in vt:
        single_line = frame.rstrip("\n")
        timestamp = float(single_line.split('"')[1])
        frameNo = int(single_line.split(".")[2])
        timing.append((frameNo, timestamp))
    timings.append(timing)

In [7]:
print("Standard deviation in frame timing:", np.std([frameNo / timestamp for frameNo, timestamp in timing[1:]]))

timing seems constant

# Show some frames

In [8]:
vidcap = cv2.VideoCapture(os.path.join(SRC, videos[0]))

In [9]:
len(timing) / 25, videos[0]

In [10]:
begin_frame = 70
step = 50
plt.figure(figsize=(15,5))
for i in range(1,6):
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, begin_frame + (i - 1) * step)
    success, image = vidcap.read()
    if success:
        ax = plt.subplot(1,5,i)
        ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        ax.set_title(f"fNo: {begin_frame + (i-1) * step}")        
plt.show()

## Light level

In [11]:
frame_light = []
for video in videos:
    vidcap = cv2.VideoCapture(os.path.join(SRC, video))
    success, image = vidcap.read()
    while success:
        frame_light.append(np.sum(image))
        success, image = vidcap.read()    

In [12]:
plt.figure(figsize=(15,5))
plt.plot(frame_light)
plt.ylabel("frame lightness")
plt.xlabel("frame number")
plt.show()

In [13]:
from scipy.signal import find_peaks

In [14]:
frame_light = np.array(frame_light)

In [15]:
light_on = find_peaks(frame_light, distance=100, height=2*(np.max(frame_light) - np.min(frame_light))/3 + np.min(frame_light))
light_off = find_peaks(-frame_light, distance=500, height=2*(np.max(-frame_light) - np.min(-frame_light))/3 + np.min(-frame_light))

In [16]:
plt.figure(figsize=(15,5))
plt.plot(frame_light, color="gray")
for ons in light_on[0]:
    plt.axvline(ons, color="green")
for offs in light_off[0]:
    plt.axvline(offs, color="red")    
plt.ylabel("frame lightness")
plt.xlabel("frame number")
plt.show()

In [17]:
light_on[0]

## Estimate  speed in pixels

In [18]:
vidcap = cv2.VideoCapture(os.path.join(SRC, videos[0]))

In [19]:
frame = 80
frame_shift = 120

In [20]:
vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)
_, a = vidcap.read()
_, a1 = vidcap.read()
vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame + frame_shift // 2)
_, b = vidcap.read()
vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame + frame_shift)
_, c = vidcap.read()

In [21]:
crop_shift = 1600
feature_shift = 74

x = 1000
y = 500
f = 110

plt.figure(figsize=(15,5))
ax = plt.subplot(131)
ax.imshow(a[:y, x:x+500,0].T, cmap="gray")
ax.axhline(f, color="red")
ax.set_title(f"{frame} - [{x}, {x + 500}]")
ax = plt.subplot(132)
ax.imshow(c[:y,x+crop_shift + feature_shift:x+crop_shift+500 + feature_shift,0].T, cmap="gray")
ax.axhline(f, color="red")
ax.set_title(f"{frame + frame_shift} - [{x+crop_shift + feature_shift}, {x + crop_shift + 500 + feature_shift}]")
ax = plt.subplot(133)
ax.imshow(b[:y,x+crop_shift // 2 + feature_shift // 2:x + crop_shift // 2 + 500 + feature_shift // 2,0].T, cmap="gray")
ax.axhline(f, color="red")
ax.set_title(f"{frame + frame_shift // 2} - [{x+crop_shift // 2 + feature_shift // 2}, {x + crop_shift // 2 + 500 + feature_shift // 2}]")
plt.show()

Posun spočtu jako:
- `frame_shift` počet framů posunu => 120
- `crop_shift` posun výřezu => 1600
- `feature_shift` počet pixelů posunu stejné featury => 74

  ```∆ = (crop_shift + feature_shift) / frame_shift```

In [22]:
shift_per_frame = (crop_shift + feature_shift) / frame_shift

In [23]:
shift_per_frame

## Angle speed

In [25]:
light_off[0]

In [26]:
frame = 135
frame_shift = 1945

vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)
_, a = vidcap.read()
vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame + frame_shift)
_, b = vidcap.read()

In [27]:
feature_shift = 9

x = 1800
y = 500
f = 300

plt.figure(figsize=(15,5))
ax = plt.subplot(121)
ax.imshow(a[:500,x:x+400,0].T, cmap="gray")
ax.axhline(f, color="red")
ax = plt.subplot(122)
ax.imshow(b[:500,x:x+400,0].T, cmap="gray")
ax.axhline(f+feature_shift, color="red")
plt.show()

In [28]:
fps = 1/timings[0][1][1]

In [29]:
pixels_per_360 = frame_shift * shift_per_frame + feature_shift
pixels_per_s = shift_per_frame * fps
s_per_360 = pixels_per_360 / pixels_per_s
frames_per_360 = np.ceil(pixels_per_360 / shift_per_frame).astype(int)

In [30]:
pixels_per_360, pixels_per_s, s_per_360, frames_per_360

## Row Stitching

In [31]:
print("Light on at: ", light_on[0])
print("Light off at: ", light_off[0])

In [32]:
rows = list(zip(light_on[0], light_off[0][1:]))

In [33]:
rows

In [34]:
plt.figure(figsize=(15,8))

# ax = plt.subplot(121)
# ax.imshow(a, cmap="gray")
M = np.float32([
            [1, 0, (-shift_per_frame)],
            [0, 1, 0]
        ])
ax = plt.subplot(111)
ax.imshow((cv2.warpAffine(a1, M, (a1.shape[1], a1.shape[0])).astype(float) + a.astype(float))[:,:,0] // 2, cmap="gray")

plt.show()

In [35]:
per_frame_width = 20 # pixels taken from one image

In [36]:
FRAME_SIZE = a1.shape[:2]

In [37]:
FRAME_SIZE

In [34]:
# TODO: create function with parameter CCW, CW

In [39]:
rows_images = []

for mn, mx in rows:
    row_image = np.zeros((FRAME_SIZE[0], np.ceil(frames_per_360 * shift_per_frame).astype(int) + FRAME_SIZE[1] // 2))
    weight_matrix = np.zeros(row_image.shape)
    
    start = mn + (mx - mn) // 2 - frames_per_360 // 2 
    end = start + frames_per_360    
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
    success, image = vidcap.read()
    
    print(f"processing frames {start}-{end}")

    for frameNo in tqdm(range(0, end - start)):
        M = np.float32([
            [1, 0, (shift_per_frame * frameNo)],
            [0, 1, 0]
        ])
        aligned_image = np.fliplr(cv2.warpAffine(image, M, (row_image.shape[1], row_image.shape[0])))
        crop_x_start = np.floor(row_image.shape[1] - image.shape[1] // 2 - per_frame_width // 2 - shift_per_frame * frameNo ).astype(int)
        crop_x_end = np.max([0, crop_x_start + per_frame_width])
        row_image[:, row_image.shape[1] - crop_x_end: row_image.shape[1] - crop_x_start] += aligned_image[:, crop_x_start:crop_x_end, 0]
        weight_matrix[:, row_image.shape[1] - crop_x_end: row_image.shape[1] - crop_x_start] += 1        
        frameNo += 1
        success, image = vidcap.read()
        
    rows_images.append(np.copy((row_image / weight_matrix)[:, image.shape[1] // 2: - per_frame_width // 2])) # crop the empty space

In [47]:
for iid, img in enumerate(rows_images):
    iio.imwrite(os.path.join(SRC, f"zavit-{iid:02d}.png"), img.astype(np.uint8))

## Load images

In [4]:
rows_images = []
for filename in sorted(os.listdir(SRC)):
    if "zavit" in filename and "png" in filename:
        rows_images.append(iio.imread(os.path.join(SRC, filename)))

In [5]:
# registration by mutual information
def mutual_information(imgA, imgB, bins=15):
    # taken from https://matthew-brett.github.io/teaching/mutual_information.html    
    hist_2d, x_edges, y_edges = np.histogram2d(
         imgA.ravel(),
         imgB.ravel(),
         bins=bins
    )
    pxy = hist_2d / float(np.sum(hist_2d))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    px_py = px[:, None] * py[None, :] # Broadcast to multiply marginals
    
    # Now we can do the calculation using the pxy, px_py 2D arrays
    nzs = pxy > 0 # Only non-zero pxy values contribute to the sum
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

def extract_images_and_compute_mi(shift, imgA, imgB, seed_position):    
    x = np.arange(seed_position[1, 0] + shift[0], seed_position[1, 0] + shift[0] + 399.5)
    y = np.arange(seed_position[1, 1] + shift[1], seed_position[1, 1] + shift[1] + 599.5)    
    xg, yg = np.meshgrid(x, y)
    interp = RegularGridInterpolator((np.arange(imgB.shape[0]), np.arange(imgB.shape[1])), imgB)
    return -mutual_information(
        imgA[seed_position[0, 0]: seed_position[0, 0] + 400, 
             seed_position[0, 1]: seed_position[0, 1] + 600].T,         
        interp((xg, yg))
    )

def to_minimize(x):
    return extract_images_and_compute_mi(shift=x, imgA=imgA, imgB=imgB, seed_position=seed_position)

def show_images(imgA, imgB, seed_position, shift):
    plt.figure(figsize=(15,5))
    ax = plt.subplot(131)
    ax.imshow(imgA[seed_position[0, 0]: seed_position[0, 0] + 400, 
                   seed_position[0, 1]: seed_position[0, 1] + 600].T, cmap="gray")
    ax.set_title("Fixed")

    x = np.arange(seed_position[1, 0] + shift[0], seed_position[1, 0] + shift[0] + 399.5)
    y = np.arange(seed_position[1, 1] + shift[1], seed_position[1, 1] + shift[1] + 599.5)    
    xg, yg = np.meshgrid(x, y)
    interp = RegularGridInterpolator((np.arange(imgB.shape[0]), np.arange(imgB.shape[1])), imgB)
    
    ax = plt.subplot(132)
    ax.imshow(interp((xg, yg)), cmap="gray")
    ax.set_title("Moved")

    ax = plt.subplot(133)
    ax.imshow(imgA[seed_position[0, 0]: seed_position[0, 0] + 400, 
                   seed_position[0, 1]: seed_position[0, 1] + 600].T + interp((xg, yg)), cmap="gray")
    ax.set_title("Blend")
    plt.show()
               

In [6]:
physics = {
    "roll":  np.round(rows_images[0].shape[1] * 0.1).astype(int) + 40,
    "scan_shift": 2160 // 2 + 30,
    "screw_thread_distance": 400,
    "first_frame": [1600, 26400]
}

In [7]:
first_frame = physics["first_frame"]
scan_shift = physics["scan_shift"]
roll = physics["roll"]

seed_position = np.array([first_frame, [first_frame[0] - scan_shift, first_frame[1] - roll]]).astype(int)
imgA = rows_images[0]
imgB = rows_images[1]
s1 = minimize(to_minimize, x0=[0, 0], method='COBYLA', bounds=[(-10, +10),(-10, 10)])

In [8]:
s1

In [9]:
show_images(imgA, imgB, seed_position, s1.x)

In [10]:
screw_thread_distance = physics["screw_thread_distance"]
seed_position = np.array([[first_frame[0] - scan_shift + 2 * screw_thread_distance, first_frame[1] - roll],
                          [first_frame[0] - 2 * scan_shift + 2 * screw_thread_distance, first_frame[1] - 2 * roll]]).astype(int)
imgA = rows_images[1]
imgB = rows_images[2]
s2 = minimize(to_minimize, x0=[0,30], method='COBYLA', bounds=[(-10, 10),(-10, 40)])

In [11]:
s2

In [12]:
show_images(imgA, imgB, seed_position, s2.x)

In [18]:
[[first_frame[0] - 2 * scan_shift + 5 * screw_thread_distance, first_frame[1] - 2 * roll], [first_frame[0] - 3 * scan_shift + 5 * screw_thread_distance, first_frame[1] - 3 * roll]]

In [60]:
seed_position = np.array([[first_frame[0] - 2 * scan_shift + 5 * screw_thread_distance, first_frame[1] - 2 * roll],
                          [first_frame[0] - 3 * scan_shift + 5 * screw_thread_distance, first_frame[1] - 3 * roll]]).astype(int)
imgA = rows_images[2]
imgB = rows_images[3]
s3 = minimize(to_minimize, x0=[0, 0], method='Powell', bounds=[(-10, 10),(-10, 40)])

In [61]:
s3

In [62]:
show_images(imgA, imgB, seed_position, s3.x)

In [55]:
[[first_frame[0] - 3 * scan_shift + 9 * screw_thread_distance, first_frame[1] - 3 * roll],
                          [first_frame[0] - 4 * scan_shift + 9 * screw_thread_distance, first_frame[1] - 4 * roll]]

In [57]:
seed_position = np.array([[first_frame[0] - 3 * scan_shift + 8 * screw_thread_distance, first_frame[1] - 3 * roll],
                          [first_frame[0] - 4 * scan_shift + 8 * screw_thread_distance, first_frame[1] - 4 * roll]]).astype(int)
imgA = rows_images[3]
imgB = rows_images[4]
s4 = minimize(to_minimize, x0=[0, 0], method='Powell', bounds=[(-10, 10),(-10, 40)])

In [58]:
s4

In [59]:
show_images(imgA, imgB, seed_position, s4.x)

In [67]:
[[first_frame[0] - 4 * scan_shift + 11 * screw_thread_distance, first_frame[1] - 4 * roll],
 [first_frame[0] - 5 * scan_shift + 11 * screw_thread_distance, first_frame[1] - 5 * roll]]

In [76]:
seed_position = np.array([[first_frame[0] - 4 * scan_shift + 11 * screw_thread_distance, first_frame[1] - 4 * roll],
                          [first_frame[0] - 5 * scan_shift + 11 * screw_thread_distance, first_frame[1] - 5 * roll]]).astype(int)
imgA = rows_images[4]
imgB = rows_images[5]
s5 = minimize(to_minimize, x0=[0, 0], method='Powell', bounds=[(-10, 10),(-10, 40)])

In [77]:
s5

In [78]:
show_images(imgA, imgB, seed_position, s5.x)

In [71]:
[[first_frame[0] - 5 * scan_shift + 13 * screw_thread_distance, first_frame[1] - 5 * roll],
 [first_frame[0] - 6 * scan_shift + 13 * screw_thread_distance, first_frame[1] - 6 * roll]]

In [73]:
seed_position = np.array([[first_frame[0] - 5 * scan_shift + 13 * screw_thread_distance, first_frame[1] - 5 * roll],
                          [first_frame[0] - 6 * scan_shift + 13 * screw_thread_distance, first_frame[1] - 6 * roll]]).astype(int)
imgA = rows_images[5]
imgB = rows_images[6]
s6 = minimize(to_minimize, x0=[0, 0], method='Powell', bounds=[(-10, 10),(-10, 40)])

In [74]:
s6

In [75]:
show_images(imgA, imgB, seed_position, s6.x)

In [84]:
shift_fixes = np.array([s1.x, s2.x, s3.x, s4.x, s5.x, s6.x])

## Merge

- sesbírat posun per row
- aggregovats `sn.x`
- provést roll každé otočky + interpolaci
- vyrobit váhovou matici pro překryv
- vynásobit / flatnout

In [81]:
shift_seed = []
shift_seed.append(np.array([first_frame, [first_frame[0] - scan_shift, first_frame[1] - roll]]).astype(int))
shift_seed.append(np.array([[first_frame[0] - scan_shift + 2 * screw_thread_distance, first_frame[1] - roll],
                            [first_frame[0] - 2 * scan_shift + 2 * screw_thread_distance, first_frame[1] - 2 * roll]]).astype(int))
shift_seed.append(np.array([[first_frame[0] - 2 * scan_shift + 5 * screw_thread_distance, first_frame[1] - 2 * roll],
                            [first_frame[0] - 3 * scan_shift + 5 * screw_thread_distance, first_frame[1] - 3 * roll]]).astype(int))
shift_seed.append(np.array([[first_frame[0] - 3 * scan_shift + 8 * screw_thread_distance, first_frame[1] - 3 * roll],
                            [first_frame[0] - 4 * scan_shift + 8 * screw_thread_distance, first_frame[1] - 4 * roll]]).astype(int))
shift_seed.append(np.array([[first_frame[0] - 4 * scan_shift + 11 * screw_thread_distance, first_frame[1] - 4 * roll],
                            [first_frame[0] - 5 * scan_shift + 11 * screw_thread_distance, first_frame[1] - 5 * roll]]).astype(int))
shift_seed.append(np.array([[first_frame[0] - 5 * scan_shift + 13 * screw_thread_distance, first_frame[1] - 5 * roll],
                            [first_frame[0] - 6 * scan_shift + 13 * screw_thread_distance, first_frame[1] - 6 * roll]]).astype(int))

In [87]:
per_row_shift = np.array([seed[0, :] - seed[1, :] - fix for seed, fix in zip(shift_seed, shift_fixes)])

In [89]:
per_row_shift

In [112]:
def real_roll(array, shift, axis=0):    
        double_image = np.concatenate([array, array], axis=1)
    interp = RegularGridInterpolator(
        (np.arange(double_image.shape[0]), np.arange(double_image.shape[1])), 
        double_image
    )
    if shift > 0:
        y = np.arange(shift, shift + array.shape[1] - 0.5, 1)
    else:
        y = np.arange(shift + array.shape[1], 2 * array.shape[1] + shift - 0.5, 1)    
    x = np.arange(array.shape[0])    
    xg, yg = np.meshgrid(x, y)    
    return interp((xg, yg)).T

In [114]:
# Test of the real roll function
plt.figure(figsize=(15,4))
ax = plt.subplot(211)
ax.imshow(real_roll(rows_images[1], per_row_shift[0,1]), cmap="gray")
ax.set_title("Rolled")
ax = plt.subplot(212)
ax.imshow(rows_images[1], cmap="gray")
ax.set_title("Original row 1")
plt.show()

In [119]:
rows_images[0].shape

In [170]:
rolled = []
# Roll all rows as required and plot them
plt.figure(figsize=(15,14))
ax = plt.subplot(7, 1, 1)
ax.imshow(rows_images[0], cmap="gray")
rolled.append(rows_images[0])
for en, row_shift in enumerate(np.cumsum(-per_row_shift[:,1])):
    ax = plt.subplot(7, 1, en+2)
    rolled.append(real_roll(rows_images[en + 1], row_shift % rows_images[en + 1].shape[1]))
    ax.imshow(rolled[-1], cmap="gray")
    ax.set_title(f"Row {en + 1}, shift: {row_shift % rows_images[en + 1].shape[1]}")
plt.show()

In [171]:
# merge rows

In [179]:
to_grid = [rolled[0]]
real_shift = [0]
for image, shift in tqdm(zip(rolled[1:], np.cumsum(per_row_shift[:,0])), total=per_row_shift.shape[0]):
    interp = RegularGridInterpolator(
        (np.arange(image.shape[0]), np.arange(image.shape[1])), 
        image
    )
    x = np.arange(image.shape[0] - 1)
    y = np.arange(image.shape[1])
    xg, yg = np.meshgrid(x,y)
    to_grid.append(interp((xg, yg)).T)
    real_shift.append(int(shift // 1))


In [194]:
out_height = rolled[0].shape[0] + np.max(real_shift)

In [195]:
full_image = np.zeros((out_height, rolled[0].shape[1], len(rolled)))
for en, (image, r_shift) in enumerate(zip(to_grid, real_shift)):
    full_image[r_shift: r_shift + image.shape[0], :, en] = image

In [248]:
blend_matrix = np.zeros((out_height, to_grid[0].shape[1], len(to_grid)))

blend_matrix[:real_shift[1], :, 0] += 1
lin_blend = np.dot(np.arange(0, 1, 1/(to_grid[0].shape[0] - real_shift[1])).reshape(-1, 1), 
                   np.ones((to_grid[0].shape[1],1)).T)
blend_matrix[real_shift[1]: to_grid[0].shape[0], :, 0] += np.flipud(lin_blend)
blend_matrix[real_shift[1]: to_grid[0].shape[0], :, 1] += lin_blend

for en, (image, r_shift) in enumerate(list(zip(to_grid, real_shift))[:-2]):
    blend_matrix[real_shift[en] + to_grid[en].shape[0]: real_shift[en + 2], :, en + 1] += 1
    lin_blend = np.dot(np.arange(0, 1, 1/(to_grid[en + 1].shape[0] + real_shift[en + 1] - real_shift[en + 2])).reshape(-1, 1), 
                       np.ones((to_grid[en + 1].shape[1],1)).T)
    blend_matrix[real_shift[en + 2]: to_grid[en + 1].shape[0] + real_shift[en + 1], :, en + 1] += np.flipud(lin_blend)
    blend_matrix[real_shift[en + 2]: to_grid[en + 1].shape[0] + real_shift[en + 1], :, en + 2] += lin_blend

blend_matrix[to_grid[en + 1].shape[0] + real_shift[en + 1]:, :, -1] += 1

In [260]:
blended_full_image = np.sum(full_image * blend_matrix, axis=2) / np.sum(blend_matrix, axis=2)

## Conclusion - Image READY

In [263]:
iio.imwrite(os.path.join(SRC, "oio.png"), blended_full_image.astype(np.uint8))

# Rotate to diff

In [269]:
edges = cv2.Canny(blended_full_image.astype(np.uint8),80,200)

In [270]:
plt.figure(figsize=(15,8))
plt.imshow(edges, cmap="gray")
plt.show()

In [285]:
lines = cv2.HoughLines(edges, 10, np.pi/1800, 1, min_theta=4/5 * np.pi, max_theta=np.pi)


In [286]:
plt.plot(lines[:,0,1])
plt.show()

### Odbočka - copak je v GPMF datech

In [106]:
import gpmf
from scipy.ndimage import gaussian_filter1d as gaussian
from scipy.signal import butter, sosfilt

In [31]:
gpmf_stream = gpmf.io.extract_gpmf_stream(os.path.join(SRC, videos[0]))

In [32]:
stream_content = []
for s in gpmf.parse.filter_klv(gpmf_stream, "GYRO"):
    content = []
    is_gyro = False
    for elt in s.value:
        content.append(elt)
        
    stream_content.append(content)

In [33]:
len(stream_content) * 207

In [35]:
len(stream_content[0])

In [50]:
plt.figure(figsize=(15,5))
ax=plt.subplot(111)
x = gaussian([np.mean([x for x, _, _ in content]) for content in stream_content], 1)
ax.plot(x - np.mean(x), label="horizontal")
y = gaussian([np.mean([x for _, x, _ in content]) for content in stream_content], 1)
ax.plot(y - np.mean(y), label="lens axis")
z = gaussian([np.mean([x for _, _, x in content]) for content in stream_content], 1)
ax.plot(z - np.mean(z), label="vertical")
plt.ylabel("rad/s")
plt.xlabel("video time (x)")
plt.legend()
plt.show()

In [51]:
from pykalman import KalmanFilter

In [89]:
kf = KalmanFilter(n_dim_obs=1, em_vars=['transition_covariance', 'observation_covariance', 'initial_state_mean'])

In [98]:
plt.plot(np.arange(0, len(y), 10), kf.em(y, n_iter=10).smooth(y[::10])[0].reshape(-1))
plt.plot(buttap(y))
plt.show()

In [161]:
sos = butter(1, 60, 'hp', fs=1000, output='sos')
filtered = sosfilt(sos, y)

In [162]:
plt.plot(filtered)
plt.plot(y)
plt.show()